# 인공지능과 가위바위보 하기

일반적인 딥러닝 기술 순서   
데이터 준비->딥러닝 네트워크 설계->학습->테스트

데이터셋   
학습용 데이터, 검증용 데이터, 시험용 데이터   
training, validatiaion, test   
   
교차 검증 특징
정보 리셋, 방식 변경
   


### 데이터 전처리   

In [ ]:
# 이미지 픽셀 값 0~255, 최소값 0, 최대값 1인지 확인   
x_train_norm, x_test_norm = x_train / 255.0, x_test / 255.0
print('최소값:',np.min(x_train_norm), ' 최대값:',np.max(x_train_norm))

사진 데이터 생성 사이트   
https://teachablemachine.withgoogle.com/
   
   
### PIL 라이브러리로 이미지 크기 조절   

In [ ]:
import os
# 이미지가 저장되어 있는 경로를 작성.
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper/scissor"
print("이미지 디렉토리 경로: ", image_dir_path)

images=glob.glob(image_dir_path + "/*.jpg")  

# 사이즈를 조절하는 위치.
target_size=(28,28)
for img in images:
    old_img=Image.open(img)
    new_img=old_img.resize(target_size,Image.ANTIALIAS)
    new_img.save(img,"JPEG")
# 제대로 작동했는지 확인.
print("가위 이미지 resize 완료!")

### 딥러닝 네트워크 설계

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

#하이퍼파라미터를 변경해서 확인. 일반적으로 높을수록 정확도가 오름.
model=keras.models.Sequential()
#(픽셀,픽셀,색) 흑백은 1, 칼라는 RGB 3
model.add(keras.layers.Conv2D(41, (3,3), activation='relu', input_shape=(28,28,3)))
model.add(keras.layers.MaxPool2D(2,2))
model.add(keras.layers.Conv2D(82, (3,3), activation='relu'))
model.add(keras.layers.MaxPooling2D((2,2)))
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(35, activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))
model.summary()

### 딥러닝 네트워크 학습

In [ ]:
model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])

model.fit(x_train, y_train, epochs=18)
#epochs는 검사횟수, 일반적으로 높을수록 정확도가 오르지만 오버피트 같은 경우 오히려 떨어지는 경우도 잦음.

### 딥러닝 테스트 데이터 생성

In [ ]:
# x_test, y_test를 만드는 방법은 x_train, y_train을 만드는 방법과 아주 유사합니다.
# [[YOUR CODE]]
def load_data(img_path):
    # 가위 : 0, 바위 : 1, 보 : 2
    number_of_data=300   # 가위바위보 이미지 개수 총합에 주의하세요.
    img_size=28
    color=3
    #흑백은 1, 칼라는 3
    #이미지 데이터와 라벨(가위 : 0, 바위 : 1, 보 : 2) 데이터를 담을 행렬(matrix) 영역을 생성합니다.
    imgs=np.zeros(number_of_data*img_size*img_size*color,dtype=np.int32).reshape(number_of_data,img_size,img_size,color)
    labels=np.zeros(number_of_data,dtype=np.int32)
​
    idx=0
    for file in glob.iglob(img_path+'/scissor/*.jpg'): #처음에 이해 못했는데 링크를 이미 앞에서 통합해둔 코드가 있음. 
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=0   # 가위 : 0
        idx=idx+1
​
    for file in glob.iglob(img_path+'/rock/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=1   # 바위 : 1
        idx=idx+1       
    
    for file in glob.iglob(img_path+'/paper/*.jpg'):
        img = np.array(Image.open(file),dtype=np.int32)
        imgs[idx,:,:,:]=img    # 데이터 영역에 이미지 행렬을 복사
        labels[idx]=2   # 보 : 2
        idx=idx+1
        
    print("학습데이터(x_test)의 이미지 개수는",idx,"입니다.")
    return imgs, labels
​
image_dir_path = os.getenv("HOME") + "/aiffel/rock_scissor_paper2"
(x_test, y_test)=load_data(image_dir_path)
x_test_1norm = x_test/255.0   # 입력은 0~1 사이의 값으로 정규화
​
print("x_test shape: {}".format(x_test.shape)) #용도에 따라 명칭을 바꿔서 작성
print("y_test shape: {}".format(y_test.shape)) #용도에 따라 명칭을 바꿔서 작성


### 테스트

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.evaluate(x_test, y_test)

### 진행과정 기록   
1. 라이브러리 이미지 크기 조절할 때 paper를 papor로 오타를 내서 에러.
2. 네트워크 설계 숫자 값에서 하이퍼 파라미터가 아닌 값을 바꿔서 에러.
3. 일단 무작정 Conv2D와 Dense값을 높여보았지만 정확도가 크게 올라가지 않음.
4. 테스트에 사용하던 데이터와 훈련에 사용하던 데이터를 맞바꿔보았지만 크게 변동은 없음.
5. slack에 올라온 데이터값을 추가하고 총값을 안 바꾼 채 10분을 테스트함.
6. epochs값을 15이상 올려도 정확도가 눈에 띄게 올라가는 경우는 없었고 오히려 떨어지는 경우가 더 많았음. 
7. 화면에서 손이 나가지 않도록 다시 촬영하고, 받은 데이터를 섞어서 총 300장으로 훈련 셋과 테스트 셋을 다시 만듬.
8. 정확도가 0.9로 높았지만 손실이 2를 넘김. 그래서 손실을 줄이려고 반복해보니 정확도만 떨어짐. 끝내 손실을 1이하로 줄이지 못함.